In [209]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 125)
pd.set_option('display.max_columns', 125)

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler

# Application Dataset

In [2]:
df_train = pd.read_csv("data/application_train.csv", index_col="SK_ID_CURR")
df_train.shape

(307511, 121)

In [3]:
df_train.head()

,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
SK_ID_CURR,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,N

In [4]:
col = df_train.columns.to_list()

In [5]:
df_train.TARGET.value_counts()

0    282686
1     24825
Name: TARGET, dtype: int64

## Missing value

In [6]:
# Persentasi missing value pada data train
total = df_train.isnull().sum().sort_values(ascending=False)
percent = (df_train.isnull().sum()/df_train.isnull().count()).sort_values(ascending=False)
missing = pd.concat([total,percent], axis=1, keys=['Total', 'Percent'])
missing

,Total,Percent
COMMONAREA_MEDI,214865,0.698723
COMMONAREA_AVG,214865,0.698723
COMMONAREA_MODE,214865,0.698723
NONLIVINGAPARTMENTS_MODE,213514,0.694330
NONLIVINGAPARTMENTS_MEDI,213514,0.694330
NONLIVINGAPARTMENTS_AVG,213514,0.694330
FONDKAPREMONT_MODE,210295,0.683862
LIVINGAPARTMENTS_MEDI,210199,0.683550
LIVINGAPARTMENTS_MODE,210199,0.683550
LIVINGAPARTMENTS_AVG,210199,0.683550


In [7]:
# drop kolom dengan missing value lebih dari 10% 
for i in range(len(missing.index)) :
    if (percent[i] >= 0.1) :
        df_train.drop(columns=[missing.index[i]], inplace=True)

        
df_train.shape

(307511, 64)

In [8]:
df_train.isnull().sum()

TARGET                            0
NAME_CONTRACT_TYPE                0
CODE_GENDER                       0
FLAG_OWN_CAR                      0
FLAG_OWN_REALTY                   0
CNT_CHILDREN                      0
AMT_INCOME_TOTAL                  0
AMT_CREDIT                        0
AMT_ANNUITY                      12
AMT_GOODS_PRICE                 278
NAME_TYPE_SUITE                1292
NAME_INCOME_TYPE                  0
NAME_EDUCATION_TYPE               0
NAME_FAMILY_STATUS                0
NAME_HOUSING_TYPE                 0
REGION_POPULATION_RELATIVE        0
DAYS_BIRTH                        0
DAYS_EMPLOYED                     0
DAYS_REGISTRATION                 0
DAYS_ID_PUBLISH                   0
FLAG_MOBIL                        0
FLAG_EMP_PHONE                    0
FLAG_WORK_PHONE                   0
FLAG_CONT_MOBILE                  0
FLAG_PHONE                        0
FLAG_EMAIL                        0
CNT_FAM_MEMBERS                   2
REGION_RATING_CLIENT        

In [104]:
# impute missing value with modus (angka sering muncul)
imp_modus = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

data = pd.DataFrame(imp_modus.fit_transform(df_train), columns=df_train.columns)
data.isnull().sum()

TARGET                         0
NAME_CONTRACT_TYPE             0
CODE_GENDER                    0
FLAG_OWN_CAR                   0
FLAG_OWN_REALTY                0
CNT_CHILDREN                   0
AMT_INCOME_TOTAL               0
AMT_CREDIT                     0
AMT_ANNUITY                    0
AMT_GOODS_PRICE                0
NAME_TYPE_SUITE                0
NAME_INCOME_TYPE               0
NAME_EDUCATION_TYPE            0
NAME_FAMILY_STATUS             0
NAME_HOUSING_TYPE              0
REGION_POPULATION_RELATIVE     0
DAYS_BIRTH                     0
DAYS_EMPLOYED                  0
DAYS_REGISTRATION              0
DAYS_ID_PUBLISH                0
FLAG_MOBIL                     0
FLAG_EMP_PHONE                 0
FLAG_WORK_PHONE                0
FLAG_CONT_MOBILE               0
FLAG_PHONE                     0
FLAG_EMAIL                     0
CNT_FAM_MEMBERS                0
REGION_RATING_CLIENT           0
REGION_RATING_CLIENT_W_CITY    0
WEEKDAY_APPR_PROCESS_START     0
HOUR_APPR_

In [105]:
data.drop(columns=['NAME_CONTRACT_TYPE', 'CODE_GENDER','WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
                  'FLAG_DOCUMENT_2', 'FLAG_DOCUMENT_3', 'FLAG_DOCUMENT_4', 'FLAG_DOCUMENT_5', 'FLAG_DOCUMENT_6', 
                  'FLAG_DOCUMENT_7', 'FLAG_DOCUMENT_8', 'FLAG_DOCUMENT_9', 'FLAG_DOCUMENT_10', 'FLAG_DOCUMENT_11', 
                  'FLAG_DOCUMENT_16', 'FLAG_DOCUMENT_15', 'FLAG_DOCUMENT_14', 'FLAG_DOCUMENT_13', 'FLAG_DOCUMENT_12', 
                  'FLAG_DOCUMENT_17', 'FLAG_DOCUMENT_18', 'FLAG_DOCUMENT_19', 'FLAG_DOCUMENT_20', 'FLAG_DOCUMENT_21', 
                  'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_CONT_MOBILE', 'ORGANIZATION_TYPE'], 
         inplace=True)

In [106]:
data.shape

(307511, 36)

## Feature encoding (cat to num)

In [107]:
enc = OrdinalEncoder()
enc.fit(data)

OrdinalEncoder()

In [109]:
enc_data = pd.DataFrame(enc.transform(data), columns=data.columns)

In [110]:
enc_data.head()

,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE
0,1.0,0.0,1.0,0.0,1754.0,1191.0,4588.0,189.0,6.0,7.0,4.0,3.0,1.0,63.0,15684.0,11935.0,12039.0,4047.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,28075.0,2.0,2.0,2.0,2.0,2638.0
1,0.0,0.0,0.0,0.0,2064.0,4338.0,6985.0,704.0,1.0,4.0,1.0,1.0,1.0,11.0,8382.0,11384.0,14501.0,5876.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85081.0,1.0,0.0,1.0,0.0,2944.0
2,0.0,1.0,1.0,0.0,338.0,228.0,662.0,30.0,6.0,7.0,4.0,3.0,1.0,47.0,6101.0,12347.0,11427.0,3636.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,72833.0,0.0,0.0,0.0,0.0,2957.0
3,0.0,0.0,1.0,0.0,1170.0,836.0,5680.0,145.0,6.0,7.0,4.0,0.0,1.0,34.0,6142.0,9533.0,5854.0,3730.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,90561.0,2.0,0.0,2.0,0.0,3155.0
4,0.0,0.0,1.0,0.0,1019.0,1635.0,3960.0,352.0,6.0,7.0,4.0,3.0,1.0,74.0,5215.0,9534.0,11376.0,2709.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,36023.0,0.0,0.0,0.0,0.0,2666.0


## Feature Scaling

In [111]:
from sklearn.preprocessing import RobustScaler

In [112]:
transformer = RobustScaler().fit(enc_data)

In [113]:
scale_data = transformer.transform(enc_data)
scale_data = pd.DataFrame(scale_data, columns=data.columns)

In [114]:
scale_data

,TARGET,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_WORK_PHONE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_2,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE
0,1.0,0.0,0.0,0.0,0.518824,-0.216151,-0.011349,-0.261780,0.0,0.00,0.0,2.0,0.0,-0.035714,0.865419,0.233104,0.156504,0.439705,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.981323,1.0,2.0,1.0,2.0,-0.290895
1,0.0,0.0,-1.0,0.0,0.883529,1.305609,0.593190,1.086387,-5.0,-0.75,-1.5,0.0,0.0,-1.892857,-0.139397,0.010117,0.606637,1.148895,0.0,1.0,0.0,0.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.222389,0.5,0.0,0.5,0.0,-0.054784
2,0.0,1.0,0.0,0.0,-1.147059,-0.681818,-1.001513,-0.678010,0.0,0.00,0.0,2.0,0.0,-0.607143,-0.453282,0.399838,0.044611,0.280341,1.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.036234,0.0,0.0,0.0,0.0,-0.044753
3,0.0,0.0,0.0,0.0,-0.168235,-0.387814,0.264061,-0.376963,0.0,0.00,0.0,-1.0,0.0,-1.071429,-0.447640,-0.738972,-0.974312,0.316789,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.338102,1.0,0.0,1.0,0.0,0.108025
4,0.0,0.0,0.0,0.0,-0.345882,-0.001451,-0.169735,0.164921,0.0,0.00,0.0,2.0,0.0,0.357143,-0.575203,-0.738567,0.035287,-0.079100,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.813497,0.0,0.0,0.0,0.0,-0.269290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0.0,0.0,-1.0,0.0,0.110588,-0.496132,0.147289,-0.547120,0.0,0.00,0.0,1.0,4.0,0.464286,0.883859,0.395386,-0.722552,0.493214,0.0,0.0,0.0,-1.0,-1.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.471066,0.0,0.0,0.0,0.0,0.373457
307507,0.0,0.0,0.0,0.0,-1.089412,-0.467118,-0.716520,-0.547120,0.0,-1.00,0.0,4.0,0.0,0.285714,-0.691207,0.491299,0.021209,-0.324157,0.0,1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.329666,0.0,0.0,0.0,0.0,0.584105
307508,0.0,0.0,0.0,0.0,0.068235,0.305609,0.280202,0.298429,0.0,0.00,-1.5,1.0,0.0,-1.714286,0.107885,-2.710239,-0.408264,-0.735169,0.0,0.0,1.0,-1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,-0.111912,3.0,0.0,3.0,0.0,-0.888889
307509,1.0,0.0,0.0,0.0,0.242353,-0.284816,-0.262295,-0.335079,0.0,-1.50,0.0,0.0,0.0,-1.607143,0.521398,-1.445973,0.355060,0.900737,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,-0.190990,0.0,0.0,0.0,0.0,0.335648


## Random undersampling data

In [115]:
X = enc_data.drop(columns="TARGET")
y = enc_data.TARGET

X.shape, y.shape

((307511, 35), (307511,))

In [116]:
undersample = RandomUnderSampler(sampling_strategy='majority')

In [117]:
X_over, y_over = undersample.fit_resample(X, y)
X_over.shape, y_over.shape

((49650, 35), (49650,))

## Training model

In [153]:
X_train, X_test, y_train, y_test = train_test_split(X_over, y_over, test_size=0.2, stratify=y_over, random_state=4)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((39720, 35), (9930, 35), (39720,), (9930,))

In [154]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(max_iter=300)

In [155]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=300)

In [156]:
y_pred = model.predict(X_test)

In [157]:
roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

0.6749210231945877

In [158]:
accuracy_score(y_test, model.predict(X_test))

0.6248741188318228

# Tuning

In [207]:
penalty = ['none', 'l2']
C = [1,2,3,4,5,6]
max_iter = [500]
solver = ['saga', 'sag', 'lbfgs']


hyperparameters = dict(penalty=penalty, C=C, max_iter=max_iter, solver=solver)
hyperparameters

{'penalty': ['none', 'l2'],
 'C': [1, 2, 3, 4, 5, 6],
 'max_iter': [500],
 'solver': ['saga', 'sag', 'lbfgs']}

In [210]:
logreg = LogisticRegression()

clf = GridSearchCV(logreg, hyperparameters, cv=5, verbose=2, n_jobs=-1)

In [211]:
#Fitting Model
best_model = clf.fit(X_train,y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


In [212]:
# Nilai hyperparameters terbaik
print('Best Penalty:', best_model.best_estimator_.get_params())

# Prediksi menggunakan model baru
y_pred = best_model.predict(X_test)

# Check performa dari model
print(classification_report(y_test, y_pred))
roc_auc_score(y_test, y_pred)

Best Penalty: {'C': 1, 'class_weight': None, 'dual': False, 'fit_intercept': True, 'intercept_scaling': 1, 'l1_ratio': None, 'max_iter': 500, 'multi_class': 'auto', 'n_jobs': None, 'penalty': 'l2', 'random_state': None, 'solver': 'lbfgs', 'tol': 0.0001, 'verbose': 0, 'warm_start': False}
              precision    recall  f1-score   support

         0.0       0.62      0.63      0.63      4965
         1.0       0.63      0.62      0.62      4965

    accuracy                           0.62      9930
   macro avg       0.62      0.62      0.62      9930
weighted avg       0.62      0.62      0.62      9930



0.6248741188318229